# Good examples and tips in Edward

Some code that works and is useful, that I didn't want to throw away.

**Note:** In many Edward tutorials they use tf.get_variable() instead of creating a tf.Variable(). When the variable doesn't already exist, and if you don't pass an initializer, then tf.get_variable() will initialize the variable using the glorot_uniform_initializer (Xavier initialization, a normal with specific mean and variance depdending on the shape...)

In [1]:
import numpy as np
import tensorflow as tf
import edward as ed
from pprint import pprint

from edward.models import Categorical, Dirichlet, Uniform, Mixture
from edward.models import Bernoulli, Normal
%matplotlib inline
import matplotlib.pyplot as plt

Instructions for updating:
Use the retry module or similar alternatives.


**Dirichlet Categorical, KLqp**

In [25]:
# code from edward/examples/dirichlet_categorical.py:
# Inferring a categorical distribution with KLqp
tf.reset_default_graph()
sess = tf.InteractiveSession()
K = 4
N = 30
# DATA
pi_true = np.random.dirichlet(np.array([20.0, 30.0, 10.0, 10.0]))
z_data = np.array([np.random.choice(K, 1, p=pi_true)[0]
                   for n in range(N)])
print("pi: {}".format(pi_true))

# MODEL
pi = ed.models.Dirichlet(tf.ones(4))
z = ed.models.Categorical(probs=pi, sample_shape=N)

# INFERENCE
qpi = ed.models.Dirichlet(tf.nn.softplus(
    tf.get_variable("qpi/concentration", [K])))

inference = ed.KLqp({pi: qpi}, data={z: z_data})
inference.run(n_iter=1500, n_samples=30)

# sess = ed.get_session()
print("Inferred pi: {}".format(sess.run(qpi.mean())))

pi: [0.32717055 0.33842328 0.17252855 0.16187761]


/Users/jeromekafrouni/.pyenv/versions/3.6.1/envs/prob-prog/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/Users/jeromekafrouni/.pyenv/versions/3.6.1/envs/prob-prog/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


1500/1500 [100%] ██████████████████████████████ Elapsed: 6s | Loss: 50.350
Inferred pi: [0.20643425 0.24809259 0.22217324 0.32329985]


In [26]:
z.shape

TensorShape([Dimension(30)])

**Linear regression, SGHMC**

In [6]:
# example from examples/bayesian_linear_regression_sghmc.py
tf.reset_default_graph()
sess = tf.InteractiveSession()
def build_toy_dataset(N, noise_std=0.5):
    X = np.concatenate([np.linspace(0, 2, num=N / 2),
                        np.linspace(6, 8, num=N / 2)])
    y = 2.0 * X + 10 * np.random.normal(0, noise_std, size=N)
    X = X.astype(np.float32).reshape((N, 1))
    y = y.astype(np.float32)
    return X, y

N = 40  # number of data points
D = 1  # number of features
# DATA
X_train, y_train = build_toy_dataset(N)
X_test, y_test = build_toy_dataset(N)

# MODEL
X = tf.placeholder(tf.float32, [N, D])
w = ed.models.Normal(loc=tf.zeros(D), scale=tf.ones(D))
b = ed.models.Normal(loc=tf.zeros(1), scale=tf.ones(1))
y = ed.models.Normal(loc=ed.dot(X, w) + b, scale=tf.ones(N))

# INFERENCE
T = 5000                        # Number of samples.
nburn = 100                     # Number of burn-in samples.
stride = 10                     # Frequency with which to plot samples.
qw = ed.models.Empirical(params=tf.Variable(tf.random_normal([T, D])))
qb = ed.models.Empirical(params=tf.Variable(tf.random_normal([T, 1])))

inference = ed.SGHMC({w: qw, b: qb}, data={X: X_train, y: y_train})
inference.run(step_size=1e-3)

/Users/jeromekafrouni/.pyenv/versions/3.6.1/envs/prob-prog/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  """
/Users/jeromekafrouni/.pyenv/versions/3.6.1/envs/prob-prog/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  


5000/5000 [100%] ██████████████████████████████ Elapsed: 8s | Acceptance Rate: 1.000


**Mixture Gaussian, MH**

In [42]:
# https://github.com/blei-lab/edward/blob/master/examples/mixture_gaussian_mh.py

**Sample shapes:** https://github.com/blei-lab/edward/issues/686

In [15]:
# DOESN'T WORK:
N = 15

cat = Categorical(probs=[0.3, 0.7], sample_shape=N)
comps = [Bernoulli(probs=[0.1, 0.9], sample_shape=N),
Bernoulli(probs=[0.9, 0.1], sample_shape=N)]

try:
    mix = Mixture(cat=cat, components=comps)
except Exception as e:
    print(e)

Shapes () and (2,) are not compatible


In [16]:
cat.shape, [c.shape for c in comps]

(TensorShape([Dimension(15)]),
 [TensorShape([Dimension(15), Dimension(2)]),
  TensorShape([Dimension(15), Dimension(2)])])

In [20]:
# WORKS:
N = 15

cat = Categorical(probs=[[0.3, 0.7]], sample_shape=N)
comps = [Bernoulli(probs=[0.1], sample_shape=N),
Bernoulli(probs=[0.9], sample_shape=N)]

mix = Mixture(cat=cat, components=comps)

In [18]:
cat.shape, [c.shape for c in comps]

(TensorShape([Dimension(15), Dimension(1)]),
 [TensorShape([Dimension(15), Dimension(1)]),
  TensorShape([Dimension(15), Dimension(1)])])

In [24]:
# WORKS:
N = 15

cat = Categorical(probs=np.array([0.3, 0.7]).reshape(1,-1), sample_shape=N)
comps = [Bernoulli(probs=[0.1], sample_shape=N),
Bernoulli(probs=[0.9], sample_shape=N)]

mix = Mixture(cat=cat, components=comps)

**Batch training**

Checkout http://edwardlib.org/tutorials/batch-training

**Shapes**

In [ ]:
N = 500  # number of data points
K = 2  # number of components
D = 2  # dimensionality of data
ed.set_seed(42)

x_train = build_toy_dataset(N)
transformedx_train = 5.0*x_train + 3.0

# Mixture model

pi = Dirichlet(tf.ones(K))
mu = Normal(tf.zeros(D), tf.ones(D), sample_shape=K)
sigmasq = InverseGamma(tf.ones(D), tf.ones(D), sample_shape=K)

cat = Categorical(probs=pi, sample_shape=N)
components = [
    MultivariateNormalDiag(mu[k], sigmasq[k], sample_shape=N)
    for k in range(K)]